In [2]:
import weaviate
from sentence_transformers import SentenceTransformer
import pandas as pd

In [4]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

D:\anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [7]:
quora_df = pd.read_csv('quora_data.csv') 

D:\anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
questions = list(quora_df["question1"])+list(quora_df["question2"])
questions = questions[0:1000]

In [19]:
def vectorize(text):
    vectors = model.encode(text)
    return vectors

In [20]:
vectorize(questions)[0:10]

array([[ 0.10344449, -0.5977836 ,  0.69776285, ...,  0.27174303,
         0.700311  ,  0.11262862],
       [-0.5434212 , -0.1582301 ,  0.29976764, ..., -0.82899505,
         0.72949684, -1.2462949 ],
       [-0.5288902 , -0.4173314 ,  0.3852845 , ..., -0.03337766,
        -0.2755678 ,  0.10695413],
       ...,
       [-0.01245438,  0.2720488 ,  0.65795195, ..., -0.31361958,
         1.0020584 , -0.3024041 ],
       [-0.3321586 ,  0.01735573,  0.08418726, ...,  0.01650712,
         0.64677864, -0.22775926],
       [-0.8335627 ,  0.00731994, -0.33935434, ...,  0.17304714,
         0.27578598, -0.44070572]], dtype=float32)

In [16]:
def init_weaviate_schema():
    schema = {
        "classes": [{
                "class": "Questions",
                "vectorizer": "none", 
                "properties": [{
                    "name": "question_text",
                    "dataType": ["text"],
                }]
        }]
    }

    client.schema.delete_all()
    client.schema.create(schema)

In [39]:
def import_posts_with_vectors(questions, vectors, batchsize=256):
    with client.batch as batch:
        for i, q in enumerate(questions):
            props = {
                "question_text": q,
            }
            batch.add_data_object(props, "Questions", vector=vectors[i])
            

D:\anaconda\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
def search(query="", limit=3):
    vec = vectorize(query)
    near_vec = {"vector": vec.tolist()}
    res = client.query.get("Questions", ["question_text", "_additional {certainty}"]).with_near_vector(near_vec).with_limit(limit).do()
    for q in res["data"]["Get"]["Questions"]:
        print("{:.4f}: {}".format(q["_additional"]["certainty"], q["question_text"]))
        print('---')

In [40]:
client = weaviate.Client("http://localhost:8080")
client.batch.configure(batch_size=100, dynamic=False,timeout_retries=3,callback=weaviate.util.check_batch_result,)
init_weaviate_schema()
vectors = vectorize(questions)
import_posts_with_vectors(questions, vectors)

In [60]:
search("How to invest in indian stock markets", 3)

0.9391: What is the step by step guide to invest in share market in india?
---
0.8450: What is the scope of supply chain management in India?
---
0.8431: What stocks are the best to invest in right now?
---
